In [3]:
# 匯入所需的庫
from bs4 import BeautifulSoup  # 用於網頁解析的 BeautifulSoup 庫
import requests  # 用於發送 HTTP 請求的 requests 庫
from PIL import Image  # 用於處理圖片的 Pillow 庫
import pytesseract  # 用於圖片文字識別的 Tesseract OCR 庫
import pandas as pd  # 用於數據處理的 pandas 庫

# 1. 從網頁中提取訊息

# 發送 HTTP 請求到 AWS 機器學習博客
page = requests.get('https://aws.amazon.com/blogs/machine-learning/')

# 確認 HTTP 狀態碼為 200，表示請求成功
if page.status_code == 200:
    # 解析網頁內容為 BeautifulSoup 物件
    soup = BeautifulSoup(page.content, 'html.parser')

    # 初始化一個列表來存放每篇文章的信息
    blog_posts = []

    # 使用 find_all() 方法找到所有文章的標籤
    for article in soup.find_all('article'):
        # 提取文章的標題
        title = article.h2.span.text if article.h2 and article.h2.span else 'N/A'
        
        # 提取文章的作者
        authors = article.footer.find_all('span', {"property":"author"}) if article.footer else []
        author_text = ', '.join([author.span.text for author in authors if author.span]) if authors else 'N/A'
        
        # 提取發佈日期
        published_date = article.time.text if article.time else 'N/A'
        
        # 提取文章的摘要
        summary = article.section.p.text if article.section and article.section.p else 'N/A'
        
        # 提取文章的超鏈接
        link = article.div.a['href'] if article.div and article.div.a else 'N/A'
        
        # 將提取的信息存入列表
        blog_posts.append([title, author_text, published_date, summary, link])

    # 將提取的文章信息轉換為 pandas DataFrame
    df = pd.DataFrame(blog_posts, columns=['Title', 'Authors', 'Published', 'Summary', 'Link'])

    # 將發佈日期轉換為 datetime 格式
    df['Published'] = pd.to_datetime(df['Published'], format='%d %b %Y', errors='coerce')

    # 調整 pandas 顯示的欄位寬度，並顯示前五行
    pd.set_option('display.max_colwidth', None)
    print(df.head())
else:
    print("網頁請求失敗，狀態碼：", page.status_code)


# 2. 從圖片中提取文本

# 設定圖片文件路徑
image_path = 'images/simple-document-image.jpg'

# 使用 Pillow 打開圖片
img = Image.open(image_path)

# 使用 Tesseract OCR 來識別圖片中的文本
text = pytesseract.image_to_string(img, lang='eng')  # lang='eng' 指定使用英文語系的 OCR 模型

# 輸出提取的文本
print("從圖片中提取的文本：\n", text)

# 如果需要提取每個字的位置數據，可以使用 image_to_data() 函數
data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

# 輸出每個字的位置信息
print("\n每個文字的位置信息：")
for i in range(len(data['level'])):
    word = data['text'][i]
    if word.strip():  # 忽略空字串
        # 輸出文字及其位置（左，上，寬，高）
        print(f"文字：{word}, 位置：({data['left'][i]}, {data['top'][i]}, {data['width'][i]}, {data['height'][i]})")


                                                                                                                           Title  \
0  Empowering everyone with GenAI to rapidly build, customize, and deploy apps securely: Highlights from the AWS New York Summit   
1                                                                    Generative AI foundation model training on Amazon SageMaker   
2                           Automate fine-tuning of Llama 3.x models with the new visual designer for Amazon SageMaker Pipelines   
3                             Implement Amazon SageMaker domain cross-Region disaster recovery using custom Amazon EFS instances   
4                                                                     Amazon Bedrock Custom Model Import now generally available   

                                                                                                  Authors  \
0                                                                                   Swami Sivasubr

In [8]:
# 匯入必要的模組
# 用於 OCR 文本識別
import pytesseract
import cv2  # 用於圖片處理
from PIL import Image  # 用於讀取圖片
import pandas as pd  # 用於將數據轉為 DataFrame

# 設定 pytesseract 路徑
# 如果 pytesseract 沒有預設安裝在環境中，需要指定路徑
# 替換為你的 tesseract 可執行文件的路徑
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# 從本地加載 employmentapp.png 圖片
# 使用本地圖片路徑
img_path = "images/employmentapp.png"
img = Image.open(img_path)

# 使用 pytesseract 對表格進行文字提取
text = pytesseract.image_to_string(img)

# 打印提取的文字
print("提取的文字內容：")
print(text)

# 假設提取的表格數據是以換行符和制表符分隔的，將其轉換為表格格式
# 這裡假設表格數據每一行用換行符分隔，列數據用制表符分隔
# 按行分割文本
rows = text.split('\n')
# 每行按制表符分割
table_data = [row.split('\t') for row in rows]

# 將數據轉換為 DataFrame 進行結構化處理
df = pd.DataFrame(table_data)

# 打印處理後的 DataFrame
print("\n轉換為 DataFrame 後的表格數據：")
print(df)

# 將 DataFrame 保存為 CSV 文件
df.to_csv('extracted_table.csv', index=False)

print("表格數據已提取並保存為 'extracted_table.csv'。")


提取的文字內容：
Employment Application
Applicant Information
Full Name: Jane Doe
Phone Number: 555-0100|
Home Address: 123 Any Street, Any Town, USA

Mailing Address: same as home address

Previous Employment History

Start Date | End Date | EmployerName __| Position Held Reason for leaving

1/15/2009 | 6/30/2011 | AnyCompany Assistant Baker | Family relocated
7/1/2011 | 8/10/2013 | AnyCompany Bread | Baker Better opportunity
8/15/2013 | present Example Corp. Head Baker N/A, current

employer



轉換為 DataFrame 後的表格數據：
                                                                            0
0                                                      Employment Application
1                                                       Applicant Information
2                                                         Full Name: Jane Doe
3                                                     Phone Number: 555-0100|
4                                 Home Address: 123 Any Street, Any Town, USA
5               